**ARIMA MODEL**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [2]:
# Load the dataset
data = pd.read_csv('/kaggle/input/netflix-stock-price-prediction/NFLX.csv', parse_dates=True, index_col='Date')
data = data['Close'].dropna()

In [3]:
# Train-test split
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

In [4]:
# Fit ARIMA model
model = ARIMA(train, order=(4, 1, 0)) 
model_fit = model.fit()

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [5]:
# Make predictions
forecast = model_fit.forecast(steps=len(test))

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [6]:
# Calculate RMSE
rmse_arima = np.sqrt(mean_squared_error(test, forecast))
print(f"ARIMA RMSE: {rmse_arima}")

ARIMA RMSE: 70.68790418035384


**LSTM Model**

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [8]:
# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

In [9]:
# Create sliding window of data
def create_dataset(data, time_step=30):
    X, y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 30
X, y = create_dataset(scaled_data, time_step)

In [10]:
# Train-test split for LSTM
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [11]:
# Reshape input data for LSTM [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [12]:
# Define the LSTM Model using PyTorch
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_1=64, hidden_layer_2=64, output_size=1, dropout=0.5):
        super(LSTMModel, self).__init__()
        # First LSTM Layer
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_1, batch_first=True)
        # Second LSTM Layer
        self.lstm2 = nn.LSTM(input_size=hidden_layer_1, hidden_size=hidden_layer_2, batch_first=True)
        # Dense Layer
        self.fc1 = nn.Linear(hidden_layer_2, 128)
        self.relu = nn.ReLU()
        # Dropout Layer
        self.dropout = nn.Dropout(dropout)
        # Output Layer
        self.fc2 = nn.Linear(128, output_size)
    
    def forward(self, x):
        # Passing through the first LSTM layer
        x, _ = self.lstm1(x)
        # Passing through the second LSTM layer
        x, _ = self.lstm2(x)
        # Taking the output of the last time step
        x = x[:, -1, :]
        # Passing through the Dense Layer
        x = self.fc1(x)
        # ReLU activation
        x = self.relu(x)
        # Dropout
        x = self.dropout(x)
        # Final Output Layer
        x = self.fc2(x)
        return x

# Hyperparameters
input_size = 1
hidden_layer_1 = 64
hidden_layer_2 = 64
output_size = 1
dropout = 0.5
epochs = 100
batch_size = 32
learning_rate = 0.001

# Initialize the model
model = LSTMModel(input_size=input_size, hidden_layer_1=hidden_layer_1, hidden_layer_2=hidden_layer_2, 
                  output_size=output_size, dropout=dropout)

In [13]:
# Loss Function and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# Convert training data to torch tensors
X_train_tensor = torch.Tensor(X_train)  # Shape should be (batch_size, sequence_length, 1)
y_train_tensor = torch.Tensor(y_train)

X_test_tensor = torch.Tensor(X_test)
y_test_tensor = torch.Tensor(y_test)

In [15]:
# Training Loop
train_losses = []
val_losses = []

for epoch in range(epochs):
    model.train()  
    optimizer.zero_grad()
    
    # Forward Pass
    output_train = model(X_train_tensor)
    
    # Compute Loss
    loss_train = criterion(output_train, y_train_tensor)
    train_losses.append(loss_train.item())
    
    # Backward Pass
    loss_train.backward()
    optimizer.step()
    
    model.eval()  
    output_val = model(X_test_tensor)
    
    #Validation Loss
    loss_val = criterion(output_val, y_test_tensor)
    val_losses.append(loss_val.item())
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss_train.item():.4f}, Validation Loss: {loss_val.item():.4f}')

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([782])) that is different to the input size (torch.Size([782, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([196])) that is different to the input size (torch.Size([196, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Train Loss: 0.0568, Validation Loss: 0.2374
Epoch [20/100], Train Loss: 0.0413, Validation Loss: 0.1640
Epoch [30/100], Train Loss: 0.0408, Validation Loss: 0.1946
Epoch [40/100], Train Loss: 0.0401, Validation Loss: 0.1515
Epoch [50/100], Train Loss: 0.0391, Validation Loss: 0.1709
Epoch [60/100], Train Loss: 0.0394, Validation Loss: 0.1700
Epoch [70/100], Train Loss: 0.0393, Validation Loss: 0.1631
Epoch [80/100], Train Loss: 0.0390, Validation Loss: 0.1702
Epoch [90/100], Train Loss: 0.0388, Validation Loss: 0.1657
Epoch [100/100], Train Loss: 0.0384, Validation Loss: 0.1682


In [16]:
# Predictions
model.eval()  

# Make Predictions on Test Data
predictions = model(X_test_tensor).detach().numpy()

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"LSTM RMSE: {rmse}")

LSTM RMSE: 0.4095831175052651
